# Topic Modelling of used functions
## Function that collects all the calls

In [16]:
import ast

class CallCollector(ast.NodeVisitor):
    def __init__(self):
        self.calls = []

    def visit_Call(self, node):
        self.calls.append(node.func.attr)
    
    def get_calls(self):
        return(self.calls) 

## Test of the function

In [15]:
tree = ast.parse('''
connection = pymysql.connect()
with connection.cursor() as cursor:
    sql = ""
    cursor.execute(sql)
    test = cursor.fetchall()
    test = pd.DataFrame(test)
''')
cc = CallCollector()
cc.visit(tree)
cc.get_calls()

['connect', 'cursor', 'execute', 'fetchall', 'DataFrame']

## Get all the code blocks from a notebook

In [17]:
from nbformat import reads, NO_CONVERT
def code_extractor(jpt):
    notebook = reads(jpt, NO_CONVERT)
    cells = notebook.cells
    code_cells = [c for c in cells if c.cell_type == 'code']
    code = []
    for cell in code_cells:
        source = cell.source
        code.append(source)
    return code

## Extract all used functions 

In [31]:
import csv
# incresing the csv field size
import ctypes
csv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))

calls = list()
with open('../data/jupyter_files.csv',"r", encoding="utf8") as csvfile:
    data = csv.DictReader(csvfile)
    for row in data:
        jpt = row['content']
        try:
            code = code_extractor(jpt)
            for c in code:
                tree = ast.parse(c)
                cc = CallCollector()
                cc.visit(tree)
                call = cc.get_calls()
                calls.append(call)
        except KeyboardInterrupt:
            break
        except:
            continue

Wall time: 0 ns


### Write the list to disk

In [ ]:
import pickle

with open('../data/calls', 'wb') as fp:
    pickle.dump(calls, fp)

### Read the list from disk

In [6]:
import pickle

with open('../data/calls', 'rb') as fp:
    calls = pickle.load(fp)

## Inspect the data and clean up

In [7]:
len(calls)

30214

### Empty Blocks

In [5]:
calls.count([])

15802

In [8]:
#Function to remove all occurences of a specific item
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
#remove all empty blocks
new_calls = remove_values_from_list(calls, [])
len(new_calls)

14412

## How many function calls are there per block?

In [7]:
import pandas as pd
c = []
for i in new_calls:
    c.append(len(i))
c = pd.Series(c)
c.describe()

count    14412.000000
mean         1.931030
std          2.260795
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         65.000000
dtype: float64

In [10]:
len(c[c > 1])

5161

In [12]:
c[c > 1].describe()

count    5161.000000
mean        3.599884
std         3.151988
min         2.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        65.000000
dtype: float64

# Topic Modelling of the functions with code blocks as a document

In [2]:
#Import all necessary libraries
import glob
import pandas as pd
import numpy as np

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')
import _pickle as pickle

import re
import random
import collections

from gensim import corpora, models
import gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from scipy import stats

pyLDAvis.enable_notebook()
random.seed(1234)

from sklearn.metrics.cluster import adjusted_rand_score
pd.options.display.max_columns = None

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
%%time
#build gensim corpus
texts = new_calls
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=3, no_above=0.4)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in texts]

Wall time: 273 ms


In [10]:
len(dictionary)

1176

In [11]:
%%time
ldamodelnormal = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=10, chunksize=100, update_every=0, alpha=1/15, random_state=1)

Wall time: 31.9 s


In [12]:
# helper to create descriptive tables (doc-topic probabilities) and visualizations for LDA models 
def getModelResults(ldamodel, corpus, dictionary):
    vis = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary, sort_topics=False)
    transformed = ldamodel.get_document_topics(corpus)
    df = pd.DataFrame.from_records([{v:k for v, k in row} for row in transformed])
    return vis, df    

In [13]:
# get the top topic per document into a list
def maxTop(x):
    mx = max(x,key=lambda item:item[1])
    if (mx[1]>0.0):
        return(mx[0])
    else:
        return 99

In [14]:
%%time
# get descriptive stuff for all models
normalv, dfnormal = getModelResults(ldamodelnormal, corpus, dictionary)

Wall time: 13.8 s


In [15]:
#print regular topics (top word probabilities)
ldamodelnormal.print_topics(num_words=8)

[(0,
  '0.063*"plot" + 0.055*"show" + 0.028*"ylabel" + 0.027*"xlabel" + 0.022*"append" + 0.020*"init" + 0.019*"figure" + 0.019*"title"'),
 (1,
  '0.058*"join" + 0.051*"extension" + 0.050*"apply" + 0.024*"execute" + 0.024*"head" + 0.018*"Series" + 0.017*"basicConfig" + 0.016*"connect"'),
 (2,
  '0.042*"count" + 0.040*"insert" + 0.037*"arange" + 0.026*"sqrt" + 0.025*"sin" + 0.024*"mean" + 0.021*"join" + 0.020*"cos"'),
 (3,
  '0.222*"read_csv" + 0.056*"add" + 0.030*"head" + 0.028*"opts" + 0.025*"plot" + 0.024*"linspace" + 0.019*"chdir" + 0.017*"json"'),
 (4,
  '0.094*"append" + 0.043*"load" + 0.023*"astype" + 0.023*"set" + 0.022*"run" + 0.022*"extension" + 0.019*"seed" + 0.019*"conv2d"'),
 (5,
  '0.121*"array" + 0.054*"DataFrame" + 0.030*"placeholder" + 0.027*"read_csv" + 0.027*"Variable" + 0.023*"drop" + 0.019*"head" + 0.018*"append"'),
 (6,
  '0.102*"head" + 0.093*"load_data" + 0.043*"reshape" + 0.038*"get" + 0.035*"split" + 0.034*"show" + 0.027*"describe" + 0.017*"DataFrame"')]

In [16]:
# inspect regular topics - ATTENTION: ALL TOPICS ARE SHIFTED WITH ID +1 w.r.t. GENSIM
normalv

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.196859 -0.048220       1        1  16.974419
1      0.008356 -0.197884       2        1  12.549253
2     -0.222872 -0.056781       3        1  12.687832
3     -0.004286  0.216316       4        1  15.472544
4      0.101873  0.035058       5        1  14.674290
5     -0.104738  0.041924       6        1  13.813716
6      0.024808  0.009588       7        1  13.827952, topic_info=     Category         Freq               Term        Total  loglift  logprob
term                                                                        
3     Default  1048.000000           read_csv  1048.000000  30.0000  30.0000
43    Default   448.000000              array   448.000000  29.0000  29.0000
2     Default   404.000000          load_data   404.000000  28.0000  28.0000
24    Default   585.000000             append   585.000000  27.0000  27.0000
27    Default   724.000000               head   724.000000  26.0000  26.0000
4     Default   424.000000               plot   424.000000  25.0000  25.0000
9     Default   257.000000                add   257.000000  24.0000  24.0000
58    Default   177.000000              apply   177.000000  23.0000  23.0000
34    Default   278.000000          extension   278.000000  22.0000  22.0000
127   Default   308.000000               join   308.000000  21.0000  21.0000
20    Default   136.000000              count   136.000000  20.0000  20.0000
6     Default   173.000000               load   173.000000  19.0000  19.0000
139   Default   170.000000            reshape   170.000000  18.0000  18.0000
64    Default   438.000000               show   438.000000  17.0000  17.0000
15    Default   141.000000             insert   141.000000  16.0000  16.0000
32    Default   325.000000          DataFrame   325.000000  15.0000  15.0000
28    Default   166.000000             arange   166.000000  14.0000  14.0000
17    Default   216.000000                get   216.000000  13.0000  13.0000
37    Default   177.000000              split   177.000000  12.0000  12.0000
148   Default   126.000000        placeholder   126.000000  11.0000  11.0000
382   Default   105.000000           Variable   105.000000  10.0000  10.0000
174   Default   142.000000             ylabel   142.000000   9.0000   9.0000
173   Default   139.000000             xlabel   139.000000   8.0000   8.0000
340   Default   130.000000           describe   130.000000   7.0000   7.0000
80    Default   111.000000               sqrt   111.000000   6.0000   6.0000
163   Default    92.000000                set    92.000000   5.0000   5.0000
33    Default   112.000000            execute   112.000000   4.0000   4.0000
124   Default    98.000000               init    98.000000   3.0000   3.0000
272   Default   138.000000             astype   138.000000   2.0000   2.0000
60    Default   101.000000               mean   101.000000   1.0000   1.0000
...       ...          ...                ...          ...      ...      ...
327    Topic7    39.566151  read_csv_metadata    54.236298   1.6631  -4.4423
630    Topic7    11.196890         coastlines    14.880837   1.6940  -5.7046
340    Topic7    89.247459           describe   130.842560   1.5959  -3.6289
37     Topic7   118.594803              split   177.292206   1.5764  -3.3446
525    Topic7    17.447102           datetime    24.082264   1.6562  -5.2611
93     Topic7    31.937862            randint    47.206821   1.5877  -4.6565
17     Topic7   126.938728                get   216.027924   1.4468  -3.2766
237    Topic7    22.727207         genfromtxt    33.277382   1.5972  -4.9967
27     Topic7   341.969482               head   724.109375   1.2283  -2.2855
221    Topic7    49.448906                use    83.793732   1.4511  -4.2193
132    Topic7    37.851929              randn    66.893425   1.4091  -4.4866
357    Topic7    20.193150             matmul    32.841736   1.4921  -5.1149
35    